In [ ]:
# -------------------------------------
# 1. Setup (Colab has TF pre-installed)
# -------------------------------------
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow version:", tf.__version__)

# -------------------------------------
# 2. Load a sample dataset (replace with your own)
# Colab provides tf.keras.datasets, here we use CIFAR-10 for demo
# -------------------------------------
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.cifar10.load_data()

# Normalize images
x_train = x_train.astype("float32") / 255.0
x_val = x_val.astype("float32") / 255.0

# Resize images to 224x224 (needed for ResNet/MobileNet)
resize_layer = tf.keras.Sequential([
    layers.Resizing(224, 224)
])

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).map(lambda x, y: (resize_layer(x), y))
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).map(lambda x, y: (resize_layer(x), y))

# -------------------------------------
# 3. Transfer Learning with ResNet50
# -------------------------------------
base_resnet = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_resnet.trainable = False  # freeze backbone

resnet_model = models.Sequential([
    base_resnet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")  # CIFAR-10 has 10 classes
])

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Training ResNet50...")
resnet_model.fit(train_ds, validation_data=val_ds, epochs=3)


# -------------------------------------
# 4. Fine-tuning (optional: unfreeze last few layers)
# -------------------------------------
base_resnet.trainable = True
for layer in base_resnet.layers[:-30]:
    layer.trainable = False

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # lower LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print("Fine-tuning ResNet50...")
resnet_model.fit(train_ds, validation_data=val_ds, epochs=2)


TensorFlow version: 2.19.0
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training ResNet50...
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 175s 102ms/step - accuracy: 0.1055 - loss: 2.3631 - val_accuracy: 0.1004 - val_loss: 2.2951
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 152s 98ms/step - accuracy: 0.1243 - loss: 2.2935 - val_accuracy: 0.1653 - val_loss: 2.2821
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 152s 98ms/step - accuracy: 0.1446 - loss: 2.2776 - val_accuracy: 0.1895 - val_loss: 2.2609
Fine-tuning ResNet50...
Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 238s 140ms/step - accuracy: 0.3179 - loss: 1.8752 - val_accuracy: 0.4812 - val_loss: 1.4853
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 206s 132ms/step - accuracy: 0.4897 - loss: 1.4367 - val_accuracy: 0.4498 - val_loss: 1.5827


In [ ]:
resnet_model.save("resnet.keras")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/cat.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted class: 4


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/deer.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predicted class: 6


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/auto.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted class: 0


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/aero.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted class: 0


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/bird.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch dimension

pred_probs = resnet_model.predict(img_array)
pred_class = np.argmax(pred_probs, axis=1)[0]

print("Predicted class:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted class: 2
